In [252]:
import pandas as pd
import numpy as np
import plotly.express as px

# Option d'affchage
pd.set_option('display.max_columns', None)

In [253]:
# Importation de la base nettoyée

df0 = pd.read_csv('data/data_cleaned.csv', sep = ',', encoding='latin-1')

In [254]:
df = df0[[
        'Orateur', 'Unnamed: 0', 'Sujet.débattu', 'Réplique', 
        'Didascalie', 'commissions', 'Groupe', 'tokenized_replique', 
        'stemmed_replique', 'tokenized_didascalie', 'stemmed_didascalie'
        ]]

df = df.rename(columns = {
                            'Unnamed: 0': 'index'
                         }
              )

### Fréquence relative des novices en nombre d’intervention et nombre de mots total

In [255]:
nb_nov = df[df['Groupe'] == 'Novice'].drop_duplicates(subset = ['Orateur'], keep = 'first').shape[0]
nb_intervention_nov  = df[df['Groupe'] == 'Novice'].shape[0]
score_nov = int(10 * nb_intervention_nov / nb_nov) / 10 

nb_exp = df[df['Groupe'] == 'Exp'].drop_duplicates(subset = ['Orateur'], keep = 'first').shape[0]
nb_intervention_exp  = df[df['Groupe'] == 'Exp'].shape[0]
score_exp = int(10 * nb_intervention_exp / nb_exp) / 10

ratio = int(10 * score_exp / score_nov) / 10

In [256]:
print('Nombre moyen de réplique par intervenant pour un Novice: ' + str(score_nov))
print('Nombre moyen de réplique par intervenant pour un Non-Novice: ' + str(score_exp))

print('Ratio : ', ratio)

Nombre moyen de réplique par intervenant pour un Novice: 460.8
Nombre moyen de réplique par intervenant pour un Non-Novice: 1021.8
Ratio :  2.2


Les non-novices parlent clairement plus que les novices : plus de 2 fois plus

#### Construction de la variable du nombre d'intervention par orateur

In [257]:
orateurs = list(set(df['Orateur']))
comptes = []

for orateur in orateurs:
    nombre_intervention = df[df['Orateur'] == orateur].shape[0]
    comptes.append((orateur, nombre_intervention))

comptes = dict(comptes)

In [258]:
df['nb_interventions'] = df.apply(lambda row: comptes[row.Orateur], axis = 1)

#### Répartition du nombre d'interventions au sein de chaque groupe

Pour les novices : 

In [259]:
px.histogram(df[df['Groupe'] == 'Novice'].drop_duplicates(subset = ['Orateur'], keep = 'first').sort_values(by = 'nb_interventions'), x = 'Orateur', y = 'nb_interventions')

Pour les non-novices : 

In [260]:
px.histogram(df[df['Groupe'] == 'Exp'].drop_duplicates(subset = ['Orateur'], keep = 'first').sort_values(by = 'nb_interventions'), x = 'Orateur', y = 'nb_interventions')

On note qu'il y a deux orateurs non-novices (R. Ferrand et F. De Rugy) qui dominent très largement le nombre d'intervention dans leur groupe. Ches les novices, il n'y a pas de telle domination.

#### Construction de la variable du nombre de mot par intervention

In [261]:
df['nb_mots'] = df.apply(lambda row: len(row.Réplique.split(' ')), axis = 1)

In [262]:
nb_mot_nov = df[df['Groupe'] == 'Novice'][['nb_mots']].sum()
score_nov = int(10 * nb_mot_nov / nb_intervention_nov) / 10 

nb_mot_exp = df[df['Groupe'] == 'Exp'][['nb_mots']].sum()
score_exp = int(10 * nb_mot_exp / nb_intervention_exp) / 10 

ratio = int(10 * score_exp / score_nov) / 10

In [263]:
print('Nombre moyen de mots par intervention pour un Novice: ' + str(score_nov))
print('Nombre moyen de mots par intervention pour un Non-Novice: ' + str(score_exp))

print('Ratio : ', ratio)

Nombre moyen de mots par intervention pour un Novice: 51.5
Nombre moyen de mots par intervention pour un Non-Novice: 46.0
Ratio :  0.8


On remarque que les interventions des novices et non-novices sont à peu près de la même longueur (légèrement plus long pour les novices).

### Analyse des didascalies

On créé pour chaque réplique les variables binaires suivantes :  
    -présence de didascalies  
    -présence de didascalies positives (applaudissement, exclamation)  
    -présence de didascalies négative 

In [264]:
# On définit d'abord les mots qui caractérisent une didascalie positive ou negative

positive_words = ['applaud', 'oui']

negative_words = ['exclam', 'protest', 'quitt', 'murmur']

def positivite_didascalie(stemmed_didascalie):
    text = stemmed_didascalie
    
    if len(text) == 0:
        return 0
    
    for word in positive_words:
        if word in text:
            return 1
    
    for word in negative_words:
        if word in text:
            return -1
        
    return 0
    

In [265]:
df['presence_didascalie'] = df.apply(lambda row: int(len(row.stemmed_didascalie) > 2), axis = 1)

df['didascalie_positive'] = df.apply(lambda row: int(positivite_didascalie(row.stemmed_didascalie) == 1), axis = 1)
df['didascalie_negative'] = df.apply(lambda row: int(positivite_didascalie(row.stemmed_didascalie) == -1), axis = 1)

Calcul de la fréquence de didascalie, de didascalie positive, et de didascalie négative dans chaque groupe. 

In [306]:
# Pour les non-novices


nb_didasc_nov = df[df['Groupe'] == 'Novice'][['presence_didascalie']].sum()
score0_nov = int(1000 * nb_didasc_nov / nb_intervention_nov) / 1000

nb_didasc_postitive_nov = df[df['Groupe'] == 'Novice'][['didascalie_positive']].sum()
score1_nov = int(1000 * nb_didasc_postitive_nov / nb_intervention_nov) / 1000

nb_didasc_negative_nov = df[df['Groupe'] == 'Novice'][['didascalie_negative']].sum()
score2_nov = int(1000 * nb_didasc_negative_nov / nb_intervention_nov) / 1000 

taux_positivite_negativite_nov = int(1000 * score1_nov / (score1_nov + score2_nov)) / 1000


# Pour les non-novices

nb_didasc_exp = df[df['Groupe'] == 'Exp'][['presence_didascalie']].sum()
score0_exp = int(1000 * nb_didasc_exp / nb_intervention_exp) / 1000

nb_didasc_postitive_exp = df[df['Groupe'] == 'Exp'][['didascalie_positive']].sum()
score1_exp = int(1000 * nb_didasc_postitive_exp / nb_intervention_exp) / 1000 

nb_didasc_negative_exp = df[df['Groupe'] == 'Exp'][['didascalie_negative']].sum()
score2_exp = int(1000 * nb_didasc_negative_exp / nb_intervention_exp) / 1000 

taux_positivite_negativite_exp = int(1000 * score1_exp / (score1_exp + score2_exp)) / 1000

In [307]:
print('Proportion des intervention avec présence de didascalies pour un novice: ' + str(score0_nov))
print('Proportion des intervention avec présence de didascalies positives pour un novice: ' + str(score1_nov))
print('Proportion des intervention avec présence de didascalies négatives pour un novice: ' + str(score2_nov))
print('Ratio de positivité des didascalies pour les novices: ' + str(taux_positivite_negativite_nov) + '\n')



print('Proportion des intervention avec présence de didascalies pour un non-novice: ' + str(score0_exp))
print('Proportion des intervention avec présence de didascalies positives pour un non-novice: ' + str(score1_exp))
print('Proportion des intervention avec présence de didascalies négatives pour un non-novice: ' + str(score2_exp))
print('Ratio de positivité des didascalies pour les non-novices: ' + str(taux_positivite_negativite_exp))

Proportion des intervention avec présence de didascalies pour un novice: 0.101
Proportion des intervention avec présence de didascalies positives pour un novice: 0.07
Proportion des intervention avec présence de didascalies négatives pour un novice: 0.019
Ratio de positivité des didascalies pour les novices: 0.786

Proportion des intervention avec présence de didascalies pour un non-novice: 0.082
Proportion des intervention avec présence de didascalies positives pour un non-novice: 0.057
Proportion des intervention avec présence de didascalies négatives pour un non-novice: 0.012
Ratio de positivité des didascalies pour les non-novices: 0.826


Conclusions :  
    -Les novices suscitent plus de réaction que les non-novices (10% contre 8%)  
    -Les réactions que suscitent les novices sont moins positives que les non-novices. Ils sont plus souvent contéstés.